## Load the required libraries

In [48]:
# Import necessary libraries
import pandas as pd
import requests
import zipfile
import io
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


## Dataset Features Explanation

- **Duration_in_month**: Duration of the credit in months.
- **Credit_history** : Credit history of the debtor.
- **Purpose**: Purpose of the credit (e.g., car, furniture, education).
- **Credit_amount**: Amount of the credit.
- **Savings_account/bonds**: Savings account/bonds status.
- **Present_employment_since**: Present employment duration.
- **Installment_rate_in_percentage_of_disposable_income**: Installment rate as a percentage of disposable income.
- **Personal_status_and_sex**: Marital status and gender.
- **Other_debtors/guarantors**: Other debtors or guarantors involved.
- **Present_residence_since**: Duration of current residence.
- **Property**: Nature of the debtor's property.
- **Age_in_years**: Debtor's age.
- **Other_installment_plans**: Other installment plans (e.g., bank, stores).
- **Housing**: Housing status (e.g., own, rent).
- **Number_of_existing_credits_at_this_bank**: Number of existing credits at the bank.
- **Job**: Nature of the debtor's job.
- **Number_of_people_being_liable_to_provide_maintenance_for**: Number of dependents.
- **Telephone**: Availability of a telephone.
- **foreign_worker**: Whether the debtor is a foreign worker.
- **Target Variable**
- **Target**: Indicates if the debtor is a good (0) or bad (1) credit risk.

## Key Takeaways
- **Comprehensive Pipeline**: Demonstrates an end-to-end machine learning workflow.
- **Model Comparison**: Cross-validation provides a robust comparison between models.
- **Hyperparameter Tuning**: Enhances model performance by finding optimal hyperparameters.

## Data Collection & Pre-Proccessing

In [52]:
# Step 1: Load the dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data'
column_names = [
     'Duration_in_month', 'Credit_history',
    'Purpose', 'Credit_amount', 'Savings_account/bonds', 'Present_employment_since',
    'Installment_rate_in_percentage_of_disposable_income', 'Personal_status_and_sex',
    'Other_debtors/guarantors', 'Present_residence_since', 'Property', 'Age_in_years',
    'Other_installment_plans', 'Housing', 'Number_of_existing_credits_at_this_bank',
    'Job', 'Number_of_people_being_liable_to_provide_maintenance_for',
    'Telephone', 'foreign_worker', 'Target'
]

# Read the data into a pandas DataFrame
df = pd.read_csv(url, sep=' ', header=None, names=column_names)
df.head(5)

Duration_in_month Credit_history Purpose  Credit_amount  \
A11                  6            A34     A43           1169   
A12                 48            A32     A43           5951   
A14                 12            A34     A46           2096   
A11                 42            A32     A42           7882   
A11                 24            A33     A40           4870   

    Savings_account/bonds Present_employment_since  \
A11                   A65                      A75   
A12                   A61                      A73   
A14                   A61                      A74   
A11                   A61                      A74   
A11                   A61                      A73   

     Installment_rate_in_percentage_of_disposable_income  \
A11                                                  4     
A12                                                  2     
A14                                                  2     
A11                                                  2     
A11                                                  3     

    Personal_status_and_sex Other_debtors/guarantors  Present_residence_since  \
A11                     A93                     A101                        4   
A12                     A92                     A101                        2   
A14                     A93                     A101                        3   
A11                     A93                     A103                        4   
A11                     A93                     A101                        4   

    Property  Age_in_years Other_installment_plans Housing  \
A11     A121            67                    A143    A152   
A12     A121            22                    A143    A152   
A14     A121            49                    A143    A152   
A11     A122            45                    A143    A153   
A11     A124            53                    A143    A153   

     Number_of_existing_credits_at_this_bank   Job  \
A11                                        2  A173   
A12                                        1  A173   
A14                                        1  A172   
A11                                        1  A173   
A11                                        2  A173   

     Number_of_people_being_liable_to_provide_maintenance_for Telephone  \
A11                                                  1             A192   
A12                                                  1             A191   
A14                                                  2             A191   
A11                                                  2             A191   
A11                                                  2             A191   

    foreign_worker  Target  
A11           A201       1  
A12           A201       2  
A14           A201       1  
A11           A201       1  
A11           A201       2

### Below Mapping indicates the real values in the column

In [22]:
# Mapping dictionaries for each categorical column
# 1. Status of existing checking account
status_mapping = {
    'A11': '< 0 DM',
    'A12': '0 <= ... < 200 DM',
    'A13': '>= 200 DM or salary assignments for at least 1 year',
    'A14': 'no checking account'
}

# 2. Credit history
history_mapping = {
    'A30': 'no credits taken/all credits paid back duly',
    'A31': 'all credits at this bank paid back duly',
    'A32': 'existing credits paid back duly till now',
    'A33': 'delay in paying off in the past',
    'A34': 'critical account/other credits existing (not at this bank)'
}

# 3. Purpose
purpose_mapping = {
    'A40': 'car (new)',
    'A41': 'car (used)',
    'A42': 'furniture/equipment',
    'A43': 'radio/television',
    'A44': 'domestic appliances',
    'A45': 'repairs',
    'A46': 'education',
    'A47': 'vacation',
    'A48': 'retraining',
    'A49': 'business',
    'A410': 'others'
}

# 4. Savings account/bonds
savings_mapping = {
    'A61': '< 100 DM',
    'A62': '100 <= ... < 500 DM',
    'A63': '500 <= ... < 1000 DM',
    'A64': '>= 1000 DM',
    'A65': 'unknown/ no savings account'
}

# 5. Present employment since
employment_mapping = {
    'A71': 'unemployed',
    'A72': '< 1 year',
    'A73': '1 <= ... < 4 years',
    'A74': '4 <= ... < 7 years',
    'A75': '>= 7 years'
}

# 6. Personal status and sex
personal_status_mapping = {
    'A91': 'male: divorced/separated',
    'A92': 'female: divorced/separated/married',
    'A93': 'male: single',
    'A94': 'male: married/widowed',
    'A95': 'female: single'
}

# 7. Other debtors/guarantors
debtors_mapping = {
    'A101': 'none',
    'A102': 'co-applicant',
    'A103': 'guarantor'
}

# 8. Property
property_mapping = {
    'A121': 'real estate',
    'A122': 'building society savings agreement/life insurance',
    'A123': 'car or other',
    'A124': 'unknown / no property'
}

# 9. Other installment plans
installment_plans_mapping = {
    'A141': 'bank',
    'A142': 'stores',
    'A143': 'none'
}

# 10. Housing
housing_mapping = {
    'A151': 'rent',
    'A152': 'own',
    'A153': 'for free'
}

# 11. Job
job_mapping = {
    'A171': 'unemployed/unskilled - non-resident',
    'A172': 'unskilled - resident',
    'A173': 'skilled employee / official',
    'A174': 'management / self-employed / highly qualified employee / officer'
}

# 12. Telephone
telephone_mapping = {
    'A191': 'none',
    'A192': 'yes, registered under the customer\'s name'
}

# 13. Foreign worker
foreign_worker_mapping = {
    'A201': 'yes',
    'A202': 'no'
}

## Text Pre-proccessing & Feature Engineering
 

**Step 2**: Define Features and Target Variable
Features (X): All columns except the 'Target' column.
Target (y): The 'Target' column indicating default status.

**Step 3**: Split the Dataset
Train-Test Split: 70% training, 30% testing.
Stratification: Ensures that the proportion of defaults and non-defaults is consistent across splits.

**Step 4**: Preprocessing
Categorical Variables: Identified as columns with data type 'object'.
Numerical Variables: Remaining columns with numeric data types.
Transformers:

- *Numerical Transformer*: Imputes missing values (if any) and scales features.

- *Categorical Transformer*: Imputes missing values (if any) and applies one-hot encoding.

- *ColumnTransformer*: Applies the appropriate transformations to each column type.


**Step 5**: Create Preprocessing Pipelines
Pipeline Integration: Ensures that preprocessing and model training are streamlined.

In [41]:
# Step 2: Define features and target variable
# Target variable adjustment: 1 (good credit), 2 (bad credit)

df['Target'] = df['Target'].map({1: 0, 2: 1})  # 0: no default, 1: default

X = df.drop('Target', axis=1)
y = df['Target']

# Step 3: Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y)

# Step 4: Preprocessing
# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Step 5: Create preprocessing pipelines
# Pipeline for numerical features (imputation and scaling)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline for categorical features (one-hot encoding)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine both transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [34]:
# Check for missing values in 'Target' before mapping
print("Number of missing values in 'Target' before mapping:")
print(df['Target'].isna().sum())

Number of missing values in 'Target' before mapping:
700


In [40]:
# Check unique values in 'Target' before mapping
print("Unique values in 'Target' before mapping:")
print(df['Target'].unique())

Unique values in 'Target' before mapping:
[0 1]


In [42]:
df.head(5)

Duration_in_month Credit_history Purpose  Credit_amount  \
A11                  6            A34     A43           1169   
A12                 48            A32     A43           5951   
A14                 12            A34     A46           2096   
A11                 42            A32     A42           7882   
A11                 24            A33     A40           4870   

    Savings_account/bonds Present_employment_since  \
A11                   A65                      A75   
A12                   A61                      A73   
A14                   A61                      A74   
A11                   A61                      A74   
A11                   A61                      A73   

     Installment_rate_in_percentage_of_disposable_income  \
A11                                                  4     
A12                                                  2     
A14                                                  2     
A11                                                  2     
A11                                                  3     

    Personal_status_and_sex Other_debtors/guarantors  Present_residence_since  \
A11                     A93                     A101                        4   
A12                     A92                     A101                        2   
A14                     A93                     A101                        3   
A11                     A93                     A103                        4   
A11                     A93                     A101                        4   

    Property  Age_in_years Other_installment_plans Housing  \
A11     A121            67                    A143    A152   
A12     A121            22                    A143    A152   
A14     A121            49                    A143    A152   
A11     A122            45                    A143    A153   
A11     A124            53                    A143    A153   

     Number_of_existing_credits_at_this_bank   Job  \
A11                                        2  A173   
A12                                        1  A173   
A14                                        1  A172   
A11                                        1  A173   
A11                                        2  A173   

     Number_of_people_being_liable_to_provide_maintenance_for Telephone  \
A11                                                  1             A192   
A12                                                  1             A191   
A14                                                  2             A191   
A11                                                  2             A191   
A11                                                  2             A191   

    foreign_worker  Target  
A11           A201       0  
A12           A201       1  
A14           A201       0  
A11           A201       0  
A11           A201       1

## Model Selection & Training

**Step 6**: Define Multiple Models

- Models included:

- Logistic Regression

- Support Vector Machine (SVM)

- Random Forest

- Naive Bayes

**Step 7**: Cross-Validation for Each Model

- *Cross-Validation*: Uses StratifiedKFold to maintain class balance.

- *Performance Metrics*: Mean accuracy and standard deviation are printed for each model.

**Step 8**: Choose the Best Model

- *Selection*: Based on cross-validation results (assumed Random Forest here).

- *Best Model*: Selected for hyperparameter tuning.

In [45]:
# Step 6: Define multiple models
models = {
    #'Logistic Regression': LogisticRegression(solver='liblinear'),
    #'SVM': SVC(),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Naive Bayes': GaussianNB()
}

# Step 7: Perform cross-validation for each model
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("Cross-Validation Results:")
for model_name, model in models.items():
    # Create a pipeline for each model
    clf = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
    
    # Perform cross-validation
    cv_scores = cross_val_score(clf, X_train, y_train, cv=kf, scoring='accuracy')
    
    print(f"Model: {model_name}")
    print(f"Mean Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})\n")

Cross-Validation Results:
Model: Random Forest
Mean Accuracy: 0.7257 (+/- 0.0266)

Model: Naive Bayes
Mean Accuracy: 0.5543 (+/- 0.1165)



## Model Pipeline & HyperParameter Tuning

- **Pipeline Creation**: Combines preprocessing and the selected model.

- **Parameter Grid:**

     -n_estimators: Number of trees in the forest.
     
     -max_depth: Maximum depth of the tree.
     
     -min_samples_split: Minimum number of samples required to split an internal node.
          
- **GridSearchCV:**

     -Cross-Validation: 5-fold stratified.
     
     -Scoring Metric: Accuracy.
     
     -Parallel Processing: n_jobs=-1 uses all available CPU cores.
     



In [50]:
best_model_name = 'Random Forest'
best_model = RandomForestClassifier(random_state=42)

# Step 9: Create a pipeline with the best model
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', best_model)])

# Define hyperparameters for tuning
param_grid = {
    'classifier__n_estimators': [50, 100, 150],
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [2, 5, 10]
}

# Perform hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

rint(f"Best Hyperparameters: {grid_search.best_params_}\n")
y_pred = grid_search.best_estimator_.predict(X_test)




Best Hyperparameters: {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 150}

Tuned Model Accuracy on Test Set: 0.7467

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.95      0.84       210
           1       0.71      0.27      0.39        90

    accuracy                           0.75       300
   macro avg       0.73      0.61      0.61       300
weighted avg       0.74      0.75      0.70       300

Confusion Matrix:
[[200  10]
 [ 66  24]]


## Evaluate Model on Test Data

In [ ]:
# Step 10: Evaluate the tuned model on the test set
p
# Final evaluation metrics
print(f"Tuned Model Accuracy on Test Set: {accuracy_score(y_test, y_pred):.4f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))